# Evaluation of Segmentation Datasets

- Thank you team for providing me the task I was able to experience the use of fiftyone which is kind of pretty kool for computer vision tasks
- I used fiftyone libraries components to complete the tasks

In [ ]:
%%capture
!pip install fiftyone shapely

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
from shapely.geometry import box
from fiftyone import ViewField as F

In [3]:
dataset_name = "open-images-apples-validset"

try:
    # Load previously downloaded dataset.
    dataset = fo.load_dataset(dataset_name).delete()
except Exception:
    dataset = foz.load_zoo_dataset(
        "open-images-v7",
        split="validation",
        label_types=["segmentations"],
        classes = ["Apple"],
        max_samples=100,
        seed=51,
        shuffle=True,
        dataset_name=dataset_name,
    )

Only found 35 (<100) samples matching your requirements
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading 'open-images-v7' split 'validation'
 100% |███████████████████| 35/35 [3.1s elapsed, 0s remaining, 11.6 samples/s]      
Dataset 'open-images-apples-validset' created


### Question 1
- Write code that finds instances where the masks overlap and visualize some positive and negative cases.

In [4]:
overlapping_samples = list()

# We will iterate over the dataset samples one by one.
for sample in dataset:
    # Obtaining the detections for each of the samples
    detections = sample.ground_truth
    
    # Obtaining the list of bounding boxes for each of the object present in an image
    bboxes = [detection.bounding_box for detection in detections.detections]
    
    # Iteration over each pair of boxes in an image and checking whether there is an overlap of any object with any other object in the whole set of objects
    for i in range(len(bboxes)):
        bbox1 = bboxes[i]
        for j in range(i+1, len(bboxes)):
            bbox2 = bboxes[j]
            
            # Convert bounding boxes to Shapely box object.
            box1 = box(bbox1[0], bbox1[1], bbox1[2], bbox1[3])
            box2 = box(bbox2[0], bbox2[1], bbox2[2], bbox2[3])
            
            # Check if the boxes overlap by Shapely box intersect method
            if box1.intersects(box2):
                overlapping_samples.append(sample.id)
                


# Id's of all the images
all_ids = set([sample.id for sample in dataset])

# Selecting Image Id's that will only contain atleast 2 or more overlapping objects
overlapping_samples = set(overlapping_samples)

# Selecting Image Id's that will only not overlapping objects
non_overlapping_samples = all_ids - overlapping_samples

# Creating a view for overlapping samples
overlaps = dataset.select(overlapping_samples)
dataset.save_view("Overlap View",overlaps)

# Creating a view for overlapping samples
non_overlaps = dataset.select(non_overlapping_samples)
dataset.save_view("Non-Overlap View",non_overlaps)

### Question 2
- Remove images that also show objects that are not labeled as apple.

In [5]:
# All image Id's
all_ids = set([sample.id for sample in dataset])
# Selecting Image Id's that contains objects other than apples
nap = dataset.select_fields("ground_truth").filter_labels('ground_truth', F("label") != 'Apple') 

# Get the IDs of images with apples
nap_ids = set([sample.id for sample in nap])
ap_ids = all_ids - nap_ids

# Creating a view for ap_ids
apples_view = dataset.select(ap_ids)
dataset.save_view("Only Apples View",apples_view)

### Viewing the saved views
- Overlap View: Images that consist of objects with masks that have overlap
- Non-Overlap View: Images that consist of objects with masks that don't have any overlap
- Only Applies View: Images that only consists of apple labeled images

In [6]:
session = fo.launch_app(dataset)

- Overlap and Non-Overlap views are not perfect, few are wrongly classified, But that was the best solution I was able to come up with 😊

### Question 3 
- Which model would you use to train an instance segmentation model for this dataset and which steps are necessary to train it?

I am considering that we need to do instance segmentation specifically for the apples
- Data Preparation: I will prepare the dataset by annotating the images with instance-level masks for each apple in the images. This will involve manually segmenting the apples and creating mask annotations for apples
- Performing Data Augmentation: We just have around 35 images, so probably using augmentation techniques like rotating, vertical or horizontal flips might be good on mask annotations and the base images
- Dataset Split: Later I will divide dataset into training and validation sets. Typically 75%:25% split ration
- Model Selection: While reading I got to know that Mask-R-CNN is a state of the art model for such tasks, I would use it and try 2 architecture variants of it, i.e. one un-trained, and one pretrained
- Model configuration: For the pretrained model, I will fine tune only the last layers, to use the pre-trained weights while training and for model trained from scratch, I will simply initiate the training. In both of them, we would like to have 1 class of apples to be segmented
- Data Loading: Loading both the image data and corresponding mask annotations for each apple in images
- Model Training: Training the Mask R-CNN model using the prepared dataset. During this training the model learns to detect and segment objects based on the provided mask annotations. The model will optimize the loss function, typically a combination of classification loss and segmentation loss
- Validation: During the training we can even pass the validation split of the data and see model's performance on this validation data
- Evaluation: We can calculate metrics like mean average precision to get to know the model's accuracy and ability to segment apples' correctly
- Model Fine-Tuning: If the model's performance is not satisfactory, we can fine-tune the model, by adjusting hyper-params
- Inference: Once the model is trained and validated, we can use it for inference on new unseen images. The trained model will detect and segment objects in the images, including apples it has been trained on